In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import requests
import json
import re
import time
import haversine as hs

In [ ]:
# Utilites stuff

# get postal code
def postal_code(address):
  matches = re.search('(\d{6})', address)
  if matches == None:
    return None

  # return a postal_code string 6 digits,
  # add 0 infront if not 6 digits
  postal_code = matches[0]
  if len(postal_code) < 6:
    postal_code = (6 - len(postal_code)) * '0' + postal_code

  return postal_code

# get postal code from OneMaps
def onemap_search(address):
  base_url = 'https://developers.onemap.sg'
  query = {
    'searchVal': address,
    'returnGeom': 'Y',
    'getAddrDetails': 'Y',
    'pageNum': '1',
  }

  request = requests.get(base_url + '/commonapi/search', params=query, timeout=5, verify=False, allow_redirects=False)
  data = json.loads(request.text)
  # print(data)

  if len(data['results']) > 0:
    results = data['results']
    return {
      'address': address,
      'postal': results[0]['POSTAL'],
      'coordinates': [
        float(results[0]['LONGTITUDE']), float(results[0]['LATITUDE']), 
      ]
    }

  else:
    return None


In [ ]:
# Data Processing - Getting Coordinates

def get_hawker_data():
  """Get hawker data from data.gov.sg"""
  hawker_resource_id = 'b80cb643-a732-480d-86b5-e03957bc82aa'
  query = {
    'resource_id': hawker_resource_id,
    'limit': 150,
  }

  url = 'https://data.gov.sg/api/action/datastore_search?{}'.format(urllib.parse.urlencode(query))
  result = []
  index = 0

  request = urllib.request.Request(url=url, headers={"User-Agent": "Mozilla/5.0"})
  fileobj = urllib.request.urlopen(request)
  data = json.loads(fileobj.read().decode('utf8'))
  hawkers = data['result']['records']
  
  for hawkerObj in hawkers:
    index = index + 1
    result.append({
      'id': index,
      'name': hawkerObj['name'],
      'address': hawkerObj['address_myenv'],
      'postalCode': postal_code(hawkerObj['address_myenv']),
      'longitude': hawkerObj['longitude_hc'],
      'latitude': hawkerObj['latitude_hc'],
    })

  print('COMPLETED: HAWKER')
  return result


def get_hdb(filePath):
  """Get HDB from preprocess hdb file"""
  df = pd.read_csv(filePath)
  uniqueAddressSet = set()
  results = []
  ONE_MAP_LIMIT = 250
  print('START: HDB')

  for i in range(len(df)):
    # remove ', Singapore' in Address and add into a set
    # this will remove repeated values inside
    uniqueAddressSet.add(df['Address'][i].replace(' ,Singapore', ''))
  
  # print(df.head())
  # print(len(df), len(uniqueAddressSet)) 
  
  uniqueAddressList = list(uniqueAddressSet)
  # communicate with onemap with array of unique addresses
  # to avoid hitting OneMap api calls, we shall wait for 20s after we hit one_map_limit
  # for i in range(len(uniqueAddressList)):
  for i in range(0, 5):
    address = uniqueAddressList[i]
    coords = onemap_search(address)

    if coords is None:
      print(address, 'is invalid')

    results.append({
      'id': i + 1,
      'name': address,
      'address': address,
      'postalCode': coords['postal'] if coords != None else '',
      'longitude': coords['coordinates'][0] if coords != None else '',
      'latitude': coords['coordinates'][1] if coords != None else '',
    })

    # prevent hitting onemap max api calls by pausing for 20s
    # can block it out if fetching of one coords is slow
    # Dont wait if we are at the last element 
    # commented out: why is python request so long?
    # if (len(results) % ONE_MAP_LIMIT == 0) and (len(results) != len(uniqueAddressList)):
    #   print("Fetched {} / {} HDB addresses so far")
    #   time.sleep(20)
  print('COMPLETED: HDB')
  return results


def get_processed_data_postal(filePath):
  """Get the long lat of the pre-processed data by the team"""
  results = []
  df = pd.read_csv(filePath)
  print('START:', filePath)

  for index in range(len(df)):
    postal = df['POSTAL_CODE'][index]
    coords = onemap_search(postal)

    if (coords == None):
       print('{} {} is invalid'.format(postal, df['NAME'][index]))

    results.append({
      'id': df['ID'][index],
      'name': df['NAME'][index],
      'address': df['ADDRESS'][index],
      'postalCode': postal,
      'longitude': coords['coordinates'][0] if coords != None else '',
      'latitude': coords['coordinates'][1] if coords != None else '',
    })

  print('COMPLETED:', filePath)
  return results
    

In [ ]:
# calculate distance of amenities from HDB

def calculate_distance(loc1, loc2):
  """Return distance between 2 coordinates in meters"""
  return hs.haversine(loc1, loc2, unit=hs.Unit.METERS)

def get_shortest_distance(hdb, amenities):
  distance = -1
  short_amenities = None
  for amenity in amenities:
    tempDistance = calculate_distance(
      (hdb['latitude'], hdb['longitude']),
      (amenity['latitude'], amenity['longitude'],)
    )

    # if distance is updated the first time
    # or the new distance calculated is smaller than current stored dsitance
    if distance == -1 or (tempDistance < distance):
      distance = tempDistance
      short_amenities = amenity

  return {
    'name': short_amenities['name'],
    'address': short_amenities['address'],
    'distance': distance
  }


def process_amenities_hdb_distance(hdb, hawker, malls, polyclinics, school, sports, transport):
  results = []

  for currentHDB in hdb:
    short_hawker = get_shortest_distance(hdb, hawker)
    short_malls = get_shortest_distance(hdb, malls)
    short_polyclinics = get_shortest_distance(hdb, polyclinics)
    short_school = get_shortest_distance(hdb, school)
    short_sports = get_shortest_distance(hdb, sports)
    short_transport = get_shortest_distance(hdb, transport)
    cbd = get_cbd_distance(hdb)

    results.push({
      'id': currentHDB['id'],
      'name': currentHDB['name'],
      'address': currentHDB['address'],
      'postalCode': currentHDB['postalCode'],
      'longitude': currentHDB['longitude'],
      'latitude': currentHDB['latitude'],

      'nearestHawkerName': short_hawker['name'],
      'nearestHawkerAddress': short_hawker['address'],
      'nearestHawkerDistance': short_hawker['distance'],

      'nearestTransportName': short_transport['name'],
      'nearestTransportAddress': short_transport['address'],
      'nearestTransportDistance': short_transport['distance'],

      'nearestMallName': short_malls['name'],
      'nearestMallAddress': short_malls['address'],
      'nearestMallDistance': short_malls['distance'],

      'nearestPolyclinicName': short_polyclinics['name'],
      'nearestPolyclinicAddress': short_polyclinics['address'],
      'nearestPolyclinicDistance': short_polyclinics['distance'],

      'nearestSchoolName': short_school['name'],
      'nearestSchoolAddress': short_school['address'],
      'nearestSchoolDistance': short_school['distance'],

      'nearestSportsName': short_sports['name'],
      'nearestSportsAddress': short_sports['address'],
      'nearestSportsDistance': short_sports['distance'],

      'distanceFromCBD': cbd,
    })
  return results


def get_cbd_distance(hdb):
  RAFFLES_PLACE_MRT = (103.8514617, 1.28412561)
  return calculate_distance((hdb['latitude'], hdb['longitude']), RAFFLES_PLACE_MRT)


In [ ]:
columns = [
  'ID', 
  'NAME', 
  'ADDRESS', 
  'POSTAL_CODE', 
  'LONGITUDE', 
  'LATITUDE', 
  ]


hawker = pd.DataFrame(get_hawker_data(), columns=columns)
hdb = pd.DataFrame(get_hdb("Resale_Price_Modified.csv"), columns=columns)
malls = pd.DataFrame(get_processed_data_postal('malls.csv'), columns=columns)
transport = pd.DataFrame(get_processed_data_postal('transport.csv'), columns=columns)
polyclinic = pd.DataFrame(get_processed_data_postal('polyclinics.csv'), columns=columns)
school = pd.DataFrame(get_processed_data_postal('primary_school.csv'), columns=columns)
sports = pd.DataFrame(get_processed_data_postal('sports_amenities.csv'), columns=columns)

hawker.to_csv('hawker_processed.csv')
hdb.to_csv('hdb_processed.csv')
malls.to_csv('malls_processed.csv')
transport.to_csv('transport_processed.csv')
polyclinic.to_csv('polyclinic_processed.csv')
school.to_csv('school_processed.csv')
sports.to_csv('sports_processed.csv')

In [ ]:
# main data processing is here

columns = [
  'id', 
  'name', 
  'address', 
  'postalCode', 
  'longitude', 
  'latitude', 
]
process_columns = [
  'ID', 
  'NAME', 
  'ADDRESS', 
  'POSTAL_CODE', 
  'LONGITUDE', 
  'LATITUDE', 
  'HAWKER_NAME', 
  'HAWKER_ADDRESS', 
  'HAWKER_DISTANCE_METERS',
  'TRANSPORT_NAME', 
  'TRANSPORT_ADDRESS', 
  'TRANSPORT_DISTANCE_METERS',
  'MALL_NAME', 
  'MALL_ADDRESS', 
  'MALL_DISTANCE_METERS',
  'POLYCLINIC_NAME', 
  'POLYCLINIC_ADDRESS', 
  'POLYCLINIC_DISTANCE_METERS',
  'PRIMARY_SCHOOL_NAME', 
  'PRIMARY_SCHOOL_ADDRESS', 
  'PRIMARY_SCHOOL_DISTANCE_METERS',
  'SPORTS_NAME', 
  'SPORTS_ADDRESS', 
  'SPORTS_DISTANCE_METERS',
  'CBD_NAME', 
  ]


hawker_pd = pd.read_csv('hawker_processed.csv', names=columns)
hdb_pd = pd.read_csv('hdb_processed.csv', names=columns)
malls_pd = pd.read_csv('malls_processed.csv', names=columns)
transport_pd = pd.read_csv('transport_processed.csv', names=columns)
polyclinic_pd = pd.read_csv('polyclinic_processed.csv', names=columns)
school_pd = pd.read_csv('school_processed.csv', names=columns)
sports_pd = pd.read_csv('sports_processed.csv', names=columns)

processed_distance = process_amenities_hdb_distance(
  hawker_pd,  
  hdb_pd, 
  malls_pd, 
  transport_pd, 
  polyclinic_pd,  
  school_pd,  
  sports_pd,  
)

processedDF = pd.DataFrame(processed_distance, columns=process_columns)
processedDF.to_csv('hdb_distance.csv')